
# Set up cấu hình

1.   Quản lý thư viện
2.   Logging set up
3.   Huggingface set up





## Quản lý thư viện

In [1]:
# BƯỚC 1: Cài đặt các thư viện cần thiết (chạy 1 lần)
!pip install --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  # GPU support
!pip install --quiet numpy pandas
!pip install --quiet transformers datasets accelerate  # Hugging Face
!pip install --quiet scikit-learn matplotlib seaborn

print("Tất cả thư viện đã được cài đặt thành công!")

Tất cả thư viện đã được cài đặt thành công!


In [2]:
# BƯỚC 2: Kiểm tra phiên bản
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import transformers
import pandas as pd
import numpy as np
import os
import logging
from datetime import datetime
from google.colab import drive
import requests
import json
from google.colab import userdata

print(f"PyTorch: {torch.__version__} | CUDA: {torch.cuda.is_available()}")
print(f"Transformers: {transformers.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

PyTorch: 2.8.0+cu126 | CUDA: False
Transformers: 4.57.1
Pandas: 2.2.2
NumPy: 2.0.2


## Logging set up

In [3]:
# Tạo thư mục log nếu chưa có
log_dir = "/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project/Logs"
os.makedirs(log_dir, exist_ok=True)

# Cấu hình logging
log_file = os.path.join(log_dir, f"ai4business_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    handlers=[
        logging.FileHandler(log_file, encoding='utf-8'),
        logging.StreamHandler()  # In ra màn hình Colab
    ]
)

## Hugging face set up

In [4]:
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    if not HF_TOKEN:
        raise ValueError("Token rỗng!")
    print("HF_TOKEN đã được tải từ Secrets")
except Exception as e:
    raise RuntimeError("Vui lòng thêm HF_TOKEN vào Colab Secrets!") from e

HF_TOKEN đã được tải từ Secrets


In [5]:
API_URL = "https://router.huggingface.co/hf-inference/models/facebook/bart-large-cnn"
headers = {"Authorization": f"Bearer {HF_TOKEN}"}

In [6]:
CACHE_DIR = "/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project/Cache"
os.makedirs(CACHE_DIR, exist_ok=True)

In [7]:
def query_hf_api_with_cache(prompt: str, day_str: str) -> str:
    """
    Gọi Hugging Face Inference API + Cache + Retry + Fallback thông minh.
    ĐÃ FIX: Lỗi "index out of range in self" → do payload sai cấu trúc.
    """
    cache_file = os.path.join(CACHE_DIR, f"summary_{day_str}.json")

    # === 1. CACHE HIT ===
    if os.path.exists(cache_file):
        try:
            with open(cache_file, 'r', encoding='utf-8') as f:
                cached = json.load(f)
            summary = cached.get("summary", "")
            if summary:
                logging.info(f"CACHE HIT | Ngày: {day_str} | Tóm tắt: {summary[:60]}...")
                return summary
        except Exception as e:
            logging.warning(f"Cache lỗi (đọc thất bại): {cache_file} | {e}")

    # === 2. GỌI API (ĐÃ SỬA PAYLOAD CHO HUGGING FACE SUMMARIZATION) ===
    logging.info(f"API CALL | Ngày: {day_str} | Gọi Hugging Face Summarization API...")

    # PAYLOAD ĐÚNG CHO MÔ HÌNH SUMMARIZATION (ví dụ: facebook/bart-large-cnn, google/pegasus-xsum)
    payload = {
        "inputs": prompt,  # Chuỗi cần tóm tắt
        "parameters": {
            "max_length": 250,
            "min_length": 100,
            "do_sample": False,
            "early_stopping": True,
            "num_beams": 4,
            "length_penalty": 2.0,
            "no_repeat_ngram_size": 3
        }
    }

    for attempt in range(1, 4):
        try:
            response = requests.post(
                API_URL,  # Đảm bảo API_URL trỏ đến endpoint summarization (không phải text-generation)
                headers=headers,
                json=payload,
                timeout=45
            )

            if response.status_code == 200:
                result = response.json()
                # ĐÚNG CẤU TRÚC: HF trả về [{"summary_text": "..."}]
                if isinstance(result, list) and len(result) > 0 and "summary_text" in result[0]:
                    summary = result[0]["summary_text"].strip()
                else:
                    summary = "Không thể trích xuất tóm tắt từ phản hồi API."

                # LƯU CACHE
                os.makedirs(CACHE_DIR, exist_ok=True)
                with open(cache_file, 'w', encoding='utf-8') as f:
                    json.dump({"summary": summary}, f, ensure_ascii=False, indent=2)

                logging.info(f"API SUCCESS | Lần {attempt} | Ngày: {day_str} | Cache lưu OK")
                return summary

            else:
                error_msg = response.text[:150]
                logging.warning(f"API ERROR | Lần {attempt} | Status {response.status_code} | {error_msg}")

        except requests.exceptions.RequestException as e:
            logging.error(f"API REQUEST FAILED | Lần {attempt} | {e}")
        except Exception as e:
            logging.error(f"API PARSE FAILED | Lần {attempt} | {e}")

# Set up dữ liệu

## Kiểm tra drive

In [8]:
# Nếu đã mount rồi thì bỏ qua, không cần mount lại
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Drive đã được mount.")

# Đường dẫn CHÍNH XÁC theo ảnh
data_dir = "/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project/Data"

# Liệt kê tất cả file trong thư mục để kiểm tra
print("Các file trong thư mục Data:")
for f in os.listdir(data_dir):
    print("  →", f)

# Đọc file với tên ĐÚNG (thêm .csv)
hotel_path = os.path.join(data_dir, "hotel_data.pkl")          # Thêm .csv
news_path = os.path.join(data_dir, "news_data.pkl")  # Đúng tên

# Kiểm tra tồn tại
if os.path.exists(hotel_path):
    print("✅ Tìm thấy: hotel_data")
else:
    print("❌ Vẫn không thấy hotel_data")

if os.path.exists(news_path):
    print("✅ Tìm thấy: news_data")
else:
    print("❌ Vẫn không thấy news_data")

Drive đã được mount.
Các file trong thư mục Data:
  → hotel_news_2024_improved_utf8bom.gsheet
  → hotel_2024_data.gsheet
  → news_data.pkl
  → hotel_data.pkl
✅ Tìm thấy: hotel_data
✅ Tìm thấy: news_data


## Kiểm tra dữ liệu

In [9]:
hotel_df = pd.read_pickle(hotel_path)
hotel_df.head()
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   day                          366 non-null    object 
 1   single_price                 366 non-null    float64
 2   single_available             366 non-null    int64  
 3   single_sold                  366 non-null    int64  
 4   double_price                 366 non-null    float64
 5   double_available             366 non-null    int64  
 6   double_sold                  366 non-null    int64  
 7   vip_price                    366 non-null    float64
 8   vip_available                366 non-null    int64  
 9   vip_sold                     366 non-null    int64  
 10  revenue                      366 non-null    float64
 11  is_weekend                   366 non-null    int64  
 12  is_holiday                   366 non-null    int64  
 13  holiday_score       

In [10]:
news_df = pd.read_pickle(news_path)
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457 entries, 0 to 456
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   day      457 non-null    object
 1   title    457 non-null    object
 2   summary  457 non-null    object
dtypes: object(3)
memory usage: 10.8+ KB


# Preprocessing -> Khởi tạo Metadata

In [11]:
class NewsSummary:
    def __init__(self, day_str, news_df):
        logging.info(f"Khởi tạo NewsSummary | Ngày đầu vào: {day_str}")

        # Chuyển ngày
        self.day = pd.to_datetime(day_str, format='%d-%m-%y', errors='coerce')
        if pd.isna(self.day):
            logging.error(f"NGÀY KHÔNG HỢP LỆ: {day_str}")
            raise ValueError(f"Ngày không hợp lệ: {day_str}")
        logging.info(f"Ngày hợp lệ: {self.day.strftime('%d-%m-%y')}")

        # Xử lý news_df
        self.news_df = news_df.copy()
        initial_count = len(self.news_df)
        self.news_df['day'] = pd.to_datetime(self.news_df['day'], format='%d-%m-%y', errors='coerce')
        self.news_df = self.news_df.dropna(subset=['day']).copy()
        dropped = initial_count - len(self.news_df)
        if dropped > 0:
            logging.warning(f"Loại bỏ {dropped} dòng ngày không hợp lệ")
        self.news_df.set_index('day', inplace=True, drop=False)

        self.have_news = self._check_news()
        self.summary_data = self._get_summary_data()
        self.prompt = self._build_prompt()
        self.final_summary = self._generate_summary_with_cache()

    def _check_news(self):
        has_news = self.day in self.news_df.index
        status = "CÓ" if has_news else "KHÔNG CÓ"
        logging.info(f"KIỂM TRA TIN TỨC | Ngày {self.day.strftime('%d-%m-%y')} | {status} tin tức")
        return has_news

    def _get_summary_data(self):
        if not self.have_news:
            logging.info("Không có tin tức → summary_data = []")
            return []
        day_data = self.news_df.loc[self.day]
        count = 1 if isinstance(day_data, pd.Series) else len(day_data)
        logging.info(f"LẤY DỮ LIỆU | {count} bài báo")
        if isinstance(day_data, pd.Series):
            return [day_data.to_dict()]
        else:
            return day_data.to_dict('records')

    def _build_prompt(self):
      if not self.summary_data or not isinstance(self.summary_data, list):
          logging.info("PROMPT | Không có dữ liệu tin tức hợp lệ")
          return "Hôm nay không có tin tức mới về thị trường khách sạn."

      article_count = len(self.summary_data)
      prompt = (
          f"Hôm nay có {article_count} tin tức nổi bật về thị trường khách sạn:\n\n"
      )

      for i, row in enumerate(self.summary_data, 1):
          title = row.get('title', 'Không có tiêu đề').strip()
          summary = row.get('summary', 'Không có tóm tắt').strip()
          if not title or not summary:
              continue  # Bỏ qua bài bị thiếu dữ liệu
          prompt += f"[{i}] {title}\nTóm tắt: {summary}\n\n"

      # === Hướng dẫn tóm tắt thông minh ===
      prompt += (
          "Yêu cầu:\n"
          "- Tóm tắt toàn bộ nội dung trên trong tối đa 250 tokens.\n"
          "- Tập trung vào: xu hướng đặt phòng, giá cả, công suất lấp đầy, chiến lược kinh doanh, tác động từ AI/đổi mới công nghệ.\n"
          "- Sử dụng giọng văn chuyên nghiệp, ngắn gọn, phù hợp báo cáo kinh doanh.\n"
          "- Cấu trúc: (1) Tổng quan thị trường, (2) 3-5 điểm nổi bật, (3) Gợi ý hành động cho doanh nghiệp khách sạn.\n"
          "- Không thêm thông tin ngoài dữ liệu đã cung cấp.\n"
      )

      logging.info(f"PROMPT | Đã tạo prompt thành công | Số bài: {article_count} | Độ dài: {len(prompt)} ký tự")
      return prompt.strip()

    def _generate_summary_with_cache(self):
        if not self.summary_data:
            return "Không có tin tức để tóm tắt."

        day_key = self.day.strftime('%d-%m-%y')
        summary = query_hf_api_with_cache(self.prompt, day_key)
        logging.info(f"TÓM TẮT HOÀN TẤT | Ngày: {day_key} | Kết quả: {summary[:60]}...")
        return summary

    def get_result(self):
        result = {
            "day": self.day.strftime('%Y-%m-%d'),
            "news_count": len(self.summary_data),
            "prompt": self.prompt,
            "ai_summary": self.final_summary
        }
        logging.info(f"GET_RESULT | Ngày: {result['day']} | {result['news_count']} bài | Tóm tắt: {result['ai_summary'][:50]}...")
        return result

In [ ]:
# ==============================
# TẠO METADATA (X) – DỰA TRÊN NewsSummary
# ==============================
X = {}

logging.info(f"Bắt đầu tạo Metadata (X) cho {len(hotel_df)} ngày...")

for i in range(len(hotel_df)):
    day_str = hotel_df.loc[i, 'day']

    try:
        # DÙNG NewsSummary → LẤY final_summary
        news_obj = NewsSummary(day_str, news_df)
        ai_news = news_obj.final_summary  # ← ĐÃ CÓ CACHE + LOG

        X[day_str] = {
            "hotel": {
                "single": {
                    "price": float(hotel_df.loc[i, 'single_price']),
                    "available": int(hotel_df.loc[i, 'single_available']),
                    "sold": int(hotel_df.loc[i, 'single_sold'])
                },
                "double": {
                    "price": float(hotel_df.loc[i, 'double_price']),
                    "available": int(hotel_df.loc[i, 'double_available']),
                    "sold": int(hotel_df.loc[i, 'double_sold'])
                },
                "vip": {
                    "price": float(hotel_df.loc[i, 'vip_price']),
                    "available": int(hotel_df.loc[i, 'vip_available']),
                    "sold": int(hotel_df.loc[i, 'vip_sold'])
                },
                "revenue": float(hotel_df.loc[i, 'revenue'])
            },
            "env": {
                "is_weekend": int(hotel_df.loc[i, 'is_weekend']),
                "is_holiday": int(hotel_df.loc[i, 'is_holiday']),
                "holiday_score": int(hotel_df.loc[i, 'holiday_score']),
                "local_event": int(hotel_df.loc[i, 'local_event']),
                "event_score": int(hotel_df.loc[i, 'event_score']),
                "weather_score": int(hotel_df.loc[i, 'weather_score']),
                "competitor_avg_single_price": float(hotel_df.loc[i, 'competitor_avg_single_price']),
                "competitor_avg_double_price": float(hotel_df.loc[i, 'competitor_avg_double_price']),
                "competitor_avg_vip_price": float(hotel_df.loc[i, 'competitor_avg_vip_price']),
                "news": ai_news
            }
        }

        if (i + 1) % 50 == 0 or i == len(hotel_df) - 1:
            print(f"Đã xử lý: {i + 1}/{len(hotel_df)} ngày")

    except Exception as e:
        logging.error(f"Lỗi ngày {day_str}: {e}")
        X[day_str] = {"error": str(e)}

# ==============================
# XUẤT FILE
# ==============================
output_path = "/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project/Output/X.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(X, f, ensure_ascii=False, indent=2)

logging.info(f"HOÀN TẤT! X.json đã được tạo | {len(X)} ngày")
print(f"\nHOÀN TẤT! File X.json đã lưu tại:\n→ {output_path}")

ERROR:root:API REQUEST FAILED | Lần 1 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:API REQUEST FAILED | Lần 2 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:API REQUEST FAILED | Lần 3 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 09-01-24: 'NoneType' object is not subscriptable
ERROR:root:API REQUEST FAILED | Lần 2 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:API REQUEST FAILED | Lần 3 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 12-01-24: 'NoneType' object is not subscriptable
ERROR:root:API REQUEST FAILED | Lần 1 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:API REQUEST FAILED | Lần 2 | HTTPSConnect

Đã xử lý: 50/366 ngày


ERROR:root:API REQUEST FAILED | Lần 1 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 22-02-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 25-02-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 27-02-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 28-02-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 01-03-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 02-03-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 07-03-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 08-03-24: 'NoneType' object is not subscriptable
ERROR:root:API REQUEST FAILED | Lần 3 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 09-03-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 10-03-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 12-03-24: 'NoneType'

Đã xử lý: 150/366 ngày


ERROR:root:API REQUEST FAILED | Lần 1 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:API REQUEST FAILED | Lần 3 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 30-05-24: 'NoneType' object is not subscriptable
ERROR:root:API REQUEST FAILED | Lần 1 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 02-06-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 04-06-24: 'NoneType' object is not subscriptable
ERROR:root:API REQUEST FAILED | Lần 1 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:API REQUEST FAILED | Lần 2 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 06-06-24: 'NoneType' object is not subscriptable
ERROR:root:API REQUEST FAILED | Lần 2 | HTTPSConn

...

Đã xử lý: 300/366 ngày


ERROR:root:API REQUEST FAILED | Lần 1 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:API REQUEST FAILED | Lần 3 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 28-10-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 03-11-24: 'NoneType' object is not subscriptable
ERROR:root:API REQUEST FAILED | Lần 1 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 05-11-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 12-11-24: 'NoneType' object is not subscriptable
ERROR:root:API REQUEST FAILED | Lần 3 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 21-11-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 26-11-24: 'NoneType' object is not subscriptable
ERROR:root:Lỗi ngày 27-11-24: 'NoneType' objec

Đã xử lý: 350/366 ngày


ERROR:root:API REQUEST FAILED | Lần 1 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:API REQUEST FAILED | Lần 3 | HTTPSConnectionPool(host='router.huggingface.co', port=443): Read timed out. (read timeout=45)
ERROR:root:Lỗi ngày 31-12-24: 'NoneType' object is not subscriptable



HOÀN TẤT! File X.json đã lưu tại:
→ /content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project/Output/X.json
